In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [2]:
df = pd.read_csv('data/secondary-prediction/secondary_prediction_players_statistics.csv')
df.groupby(['game']).size().sort_values(ascending=True)

game
ESPORTSTMNT01_2690210    2
ESPORTSTMNT03_2564748    2
ESPORTSTMNT03_2564761    2
ESPORTSTMNT03_2564782    2
ESPORTSTMNT03_2564812    2
                        ..
ESPORTSTMNT01_2781982    2
ESPORTSTMNT01_2781984    2
ESPORTSTMNT01_2781985    2
ESPORTSTMNT01_2781743    2
NA1_4217912820           2
Length: 2605, dtype: int64

In [3]:
y = df['firstTower'].copy()
X = df.drop(['game', 'firstBlood', 'kills', 'deaths', 'firstTower', 'firstHerald', 'dragons', 'barons', 'inhibitors', 'towers', 'heralds'], axis=1)
X

,flagSide,topGP,topWR,topKDA,jungleGP,jungleWR,jungleKDA,midGP,midWR,midKDA,carryGP,carryWR,carryKDA,suppGP,suppWR,suppKDA
0,1,3,0.67,4.0,0,0.00,0.0,12,0.42,4.2,1,0.00,4.0,1,1.00,9.0
1,0,6,0.50,5.1,4,0.50,4.7,0,0.00,0.0,1,0.00,1.3,11,0.36,2.4
2,1,4,0.25,2.2,2,0.00,1.2,3,0.33,4.1,0,0.00,0.0,0,0.00,0.0
3,0,4,0.00,1.5,3,0.33,1.1,2,0.50,4.2,4,0.50,3.0,0,0.00,0.0
4,1,3,0.00,1.6,0,0.00,0.0,1,0.00,0.7,5,0.40,3.1,0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,0,9,0.33,1.8,7,0.57,4.1,24,0.67,4.2,33,0.48,3.9,25,0.48,3.4
5206,1,10,0.90,4.6,1,1.00,6.0,3,0.67,4.3,16,0.63,4.7,7,0.71,5.1
5207,0,4,0.50,1.7,8,0.50,3.2,3,0.00,2.8,2,0.50,3.2,6,0.67,3.3
5208,1,1,0.00,1.2,0,0.00,0.0,0,0.00,0.0,1,1.00,9.5,4,0.25,2.5


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42, stratify=y)
    scaler = StandardScaler()   
    print(X_train) 
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test
X_train,X_test,y_train,y_test = preprocess_input(X,y)

      flagSide  topGP  topWR  topKDA  jungleGP  jungleWR  jungleKDA  midGP  \
2398         1      2   1.00    15.0         4      0.75       10.2      5   
5134         1      4   0.50     3.5        11      0.36        3.7      1   
4239         0      2   0.50     4.2         1      0.00        0.3      3   
3572         1      3   0.67     3.8         4      0.50        5.2      6   
1503         0      1   1.00     6.0         2      0.50        3.9      0   
...        ...    ...    ...     ...       ...       ...        ...    ...   
4861         0      1   0.00     1.5        19      0.47        3.1     11   
2314         1      0   0.00     0.0        11      0.36        2.8     19   
878          1      8   0.25     2.2         0      0.00        0.0      4   
4751         0      2   0.50     3.0         1      0.00        1.0      2   
1165         0     22   0.64     4.5         2      0.00        1.6      1   

      midWR  midKDA  carryGP  carryWR  carryKDA  suppGP  suppWR

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine (Linear Kernel)': LinearSVC(),
    'Support Vector Machine (RBF Kernel)': SVC(),
    'Decission Tree': DecisionTreeClassifier(),
    'Adaboost': AdaBoostClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting Classifier': GradientBoostingClassifier()
}

GBCmodel = GradientBoostingClassifier()
GBCmodel.fit(X_train,y_train)


for name, model in models.items():
  model.fit(X_train,y_train)
  print(name + ' trained')

Logistic Regression trained


C:\Users\linco\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Support Vector Machine (Linear Kernel) trained
Support Vector Machine (RBF Kernel) trained
Decission Tree trained
Adaboost trained
Random Forest trained
Gradient Boosting Classifier trained


In [6]:
scores_list = []

for name,model in models.items():    
    scores_list.append({
    'Model': name,
    'Accuracy': accuracy_score(y_test,model.predict(X_test)),
    'Precision':  precision_score(y_test,model.predict(X_test)),
    'Recall': recall_score(y_test,model.predict(X_test)),
    'F1-Score': f1_score(y_test,model.predict(X_test))
    })
scores = pd.DataFrame(scores_list)

In [7]:
scores

,Model,Accuracy,Precision,Recall,F1-Score
0,Logistic Regression,0.547025,0.548708,0.529750,0.539063
1,Support Vector Machine (Linear Kernel),0.547985,0.549603,0.531670,0.540488
2,Support Vector Machine (RBF Kernel),0.559501,0.560311,0.552783,0.556522
3,Decission Tree,0.509597,0.509615,0.508637,0.509126
4,Adaboost,0.557582,0.559761,0.539347,0.549365
5,Random Forest,0.555662,0.555556,0.556622,0.556088
6,Gradient Boosting Classifier,0.540307,0.540541,0.537428,0.538980
